In this project, we are building a multi-agent system using the crewai library to perform market research on emerging trends in the restaurant industry. The focus areas are digital menus, AI in restaurants, and pricing strategies. The system consists of two main agents:

* Trend Research Agent: Gathers relevant data on specified topics.
* Market Researcher Agent: Synthesizes the gathered information into a coherent report.

# Setup

First, we install the necessary libraries and retrieve the API keys required for accessing OpenAI's language models and performing web searches.

In [ ]:
# Install the 'crewai' library and its tools, along with 'openai' and 'serper' for search capabilities
!pip install crewai
!pip install 'crewai[tools]'
!pip install openai
!pip install serper

INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 530.7 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of mem0ai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=a4197cad3c2288b1c82dab3d15931a59b8934b0af2d08435d7e2790ecd373d45
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
  Attempting uninstall: pytest
    Fou

In [ ]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('genai_course')  # OpenAI API key for language models
serper_api_key = userdata.get('serper_api')    # Serper API key for web search capabilities

We are working in a Google Colab environment and need to install the required packages. The crewai library allows us to create and manage AI agents. The serper library provides web search functionality. API keys are securely retrieved using userdata.get() to ensure sensitive information is not exposed.

Next, we import the essential libraries for agent creation and set the API keys as environment variables.

In [ ]:
# Import essential libraries for agent creation and web tools
import os
from crewai import Agent, Task, Crew, Process          # Core components for creating agents and tasks
from crewai_tools import SerperDevTool, ScrapeWebsiteTool  # Tools for web searching and scraping
from IPython.display import display, Markdown          # For displaying outputs in a Jupyter notebook
from langchain_openai import ChatOpenAI

In [ ]:
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

By setting the API keys as environment variables, we ensure that they are accessible to any part of the program that requires them, such as the language model interface and web tools.

# Tools

We define the tools that our agents will use to perform web searches and scrape website content.

In [ ]:
# Initialize the tool for performing web searches using Serper API
search_tool = SerperDevTool()

# Initialize the tool for scraping website content
scrape_tool = ScrapeWebsiteTool()

The SerperDevTool allows agents to perform web searches, while the ScrapeWebsiteTool enables them to extract content from websites. These tools expand the agents' capabilities beyond text generation to include data retrieval from the internet.

# Research AI Agent + Task

We define the Trend Research Agent responsible for gathering relevant data.

In [ ]:
# Define the Research AI Agent with its role, goal, and backstory
research_agent = Agent(
    role="Trend Research Agent",
    goal="Retrieve relevant data on digital menus, AI in restaurants, and pricing strategies",
    backstory="You are responsible for gathering relevant information on key trends in the restaurant industry for 2024 and beyond",
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4o model
    tools=[search_tool, scrape_tool],  # Assigning the search and scrape tools
    verbose=True
)

The agent's role, goal, and backstory provide context that guides its behavior. By specifying the language model and tools, we define how the agent will perform its tasks. The temperature parameter in the language model influences the creativity of the output.

Tasks are assigned to agents, meaning that one agent can have multiple tasks.

In the crewai framework, agents can handle multiple tasks. This allows for efficient use of resources and enables agents to perform related tasks without the need for additional agents.

We create two tasks for the Trend Research Agent:

1. Trend Task: Research emerging trends in digital menus.
2. Pricing Task: Research new pricing strategies for restaurants.

In [ ]:
# Create a task for researching trends in digital menus
trend_task = Task(
    description="Research emerging trends in 2024 and 2025 on digital menus",
    expected_output="All the information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the research agent
)

In [ ]:
# Create a task for researching pricing strategies
pricing_task = Task(
    description="Research new pricing strategies for restaurants for takeaway and delivery",
    expected_output="All the retrieved information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the same agent
)

By assigning both tasks to the research_agent, we leverage its capabilities to gather all necessary data. The expected_output field defines what we anticipate receiving from the agent after task completion.

# Writer Agent

We define the Market Researcher Agent responsible for synthesizing the data into a coherent report.

In [ ]:
# Define the writer agent to generate reports
writer_agent = Agent(
    role="Market Researcher that provides reports",
    goal="Generate reports based on search results",
    backstory="You are an experienced Market Researcher that synthesizes search results into a coherent report",
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
    verbose=True
)

The writer_agent is designed to process the data collected by the research_agent and compile it into a structured report. The agent's role and backstory help guide its behavior during the task.

We define the writing task for the writer_agent, using the outputs of the previous tasks as context.



In [ ]:
# Define the writing task
writing_task = Task(
    description="Generate a report based on the retrieved search data.",
    expected_output="A detailed and coherent report in Markdown on the trends in menu design, AI in restaurants, and pricing strategies. The report must start with an executive summary, followed by the content per topic, and conclude with actionable recommendations",
    agent=writer_agent,
    context=[trend_task, pricing_task]  # Using previous tasks as context
)

The context parameter allows the writer_agent to access the outputs from the trend_task and pricing_task. This is essential for the agent to incorporate the research findings into the final report.

# Assemble the team of AI Agents

We create a crew to manage the agents and tasks, specifying that tasks should be executed sequentially.

In [ ]:
# Create the AI team for market research
crew = Crew(
    agents=[research_agent, writer_agent],  # List of agents involved
    tasks=[trend_task, pricing_task, writing_task],  # List of tasks to execute
    verbose=True,
    process=Process.sequential  # Ensuring tasks are processed in order
)
result = crew.kickoff()  # Start the task execution

# Agent: Trend Research Agent
## Task: Research emerging trends in 2024 and 2025 on Digital Menus


# Agent: Trend Research Agent
## Thought: To fulfill the task of researching emerging trends in 2024 and 2025 on digital menus, I need to gather information from credible sources online. I will begin by searching for recent articles or reports related to digital menu trends for the upcoming years.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"emerging trends in digital menus 2024 2025\"}"
## Tool Output: 

Search results: Title: 5 Biggest Restaurant Technology Trends for 2025 - Craver
Link: https://www.craverapp.com/blog/restaurant-technology-trends
Snippet: Get a sneak peek at the hottest restaurant technology trends for 2025 — from AI solutions to hyper-personalization ordering, ...
---
Title: 10 Restaurant Technology Trends to Watch in 2025 | NetSuite
Link: https://www.netsuite.com/portal/resource/articles/business-strategy/restaurant-technology-trends.shtml

The Crew object orchestrates the agents and tasks. By setting process=Process.sequential, we ensure that tasks are completed in order, which is important when tasks depend on the outputs of previous tasks.

**ReACT prompt engineering technique**
ReACT (Reasoning and Acting) is a prompt engineering technique where the AI agent is designed to reason through tasks and decide on actions. It involves generating reasoning traces and actions that guide the agent towards the desired outcome. This technique improves the agent's ability to handle complex tasks by combining reasoning with action.

Finally, we extract and display the outputs from each task in a readable format using Markdown. We iterate over the task outputs and use Markdown to format the display. This helps in presenting the results in a structured and readable manner within a Jupyter notebook.

In [ ]:
# Extract and display each agent's output in Markdown
for idx, task_output in enumerate(result.tasks_output):
  display(Markdown(f"### agent {idx +1}: task_output.agent]\n{task_output.raw}"))

### agent 1: task_output.agent]
Digital Menu Boards in 2024 – Top Trends You Need To Know

The digital menu board is not just a modern convenience; it’s a dynamic tool that’s reshaping the customer experience in the food service industry. As we look towards 2024, several key trends are emerging that restaurant owners and operators must be aware of to stay competitive. Here are some of the key trends we see as important to success in the coming year:

1. **Deeper Integration with Back-Office Data and POS Systems**
   - The integration of digital menu boards with back-office data and POS systems is becoming more sophisticated, ensuring that menu offerings, prices, and stock levels are updated in real-time. This streamlines operations and enhances the accuracy of digital displays. In 2024, expect these integrations to be standard practice.

2. **Strategic Use of Analytics to Optimize Menu Marketing**
   - Analytics are crucial in understanding customer preferences and behavior, allowing restaurants to strategically market items on their digital menu boards. In 2024, analytics will be more deeply embedded, enabling more targeted marketing campaigns.

3. **Lower Initial Investment Than Ever Before**
   - The cost of digital menu board technology is decreasing, making it more accessible for smaller establishments. This trend will continue in 2024, with the initial investment becoming more manageable for a broader range of businesses.

4. **Easier Operations with Smarter Management Tools**
   - Systems like SmarterSign are making it easier for restaurant operators to control their digital menu boards, with advanced design tools that enable rapid and accurate menu creation without design experience.

5. **Dynamic Presentations with AI Integration**
   - AI is set to enhance digital menu boards by providing real-time adaptability based on customer demographics, purchase history, and external data such as weather conditions. This trend towards dynamic presentations means digital menu boards will actively engage with customers, providing a tailored experience to drive sales and enhance the dining experience.

These trends highlight how digital menu boards are becoming an integral part of the dining experience, with advancements in integration, analytics, affordability, management, and AI-driven personalization. For restaurant owners, keeping abreast of these trends is essential to remain competitive in an increasingly technology-driven market.

### agent 2: task_output.agent]
Menu Pricing Strategies You Need In 2024

The restaurant industry is markedly shifting towards digital integration, as evidenced by Dineout’s findings: 77% of customers prefer dining out and love digital menus, and 23% lean towards home delivery and takeaways. A notable example is Burger King’s implementation of digital menu boards across 6,500 U.S. outlets within four months, showcasing the effective use of digital technology to enhance customer engagement and operational efficiency.
This blog will explain essential menu pricing strategies for restaurant owners focusing on integrating digital menus: relative pricing techniques, cost-plus pricing strategies, adjustments based on restaurant types, and premium pricing for exotic cuisines. 

Implement Dining with Digital Menus
Digital menus have transformed the typical restaurant experience, shifting from traditional service to a more interactive and efficient model. These menus offer a visual feast, ensure order accuracy, track food preparation, and allow real-time updates. This technological shift means staff can focus more on engaging with customers than merely taking orders.
Consumers, already accustomed to using digital technology for reviews and reservations, increasingly embrace digital ordering, with studies showing a significant percentage opting for digital takeout options. 
A pivotal case study from Chili reveals that implementing digital tablets led to a 20% surge in appetizer and dessert sales, demonstrating the potential for increased revenue and customer engagement. Overall, restaurants transitioning to digital menus report an average sales increase of 10-15%, showcasing the substantial impact of this innovative approach on the restaurant industry.

Leverage Relative Menu Pricing Strategy
According to a study, a relative menu pricing strategy can increase the sales of high-profit items by 13% and the overall revenue by 4.5%.
The relative menu pricing strategy can be an intelligent way to enhance profits while catering to diverse customer preferences. For example, an Indian restaurant specializes in regional cuisines. The menu can feature a high-margin item like Paneer Butter Masala at ₹250 alongside a lower-margin, upscale option like Lobster Curry at ₹450 and a moderately priced, high-margin dish like Chicken Tikka Masala at ₹300.
Budget-conscious diners favor Paneer Butter Masala, while those seeking luxury choose Lobster Curry. Chicken Tikka Masala, priced in between, entices customers willing to spend more than Paneer Butter Masala but less than Lobster Curry. This approach steers customers toward high-margin dishes by positioning them near a pricier option, making them appear affordable and attractive. It accommodates various budgets while optimizing profit.

Implement Cost-Plus Pricing Strategy 
The Cost-Plus Pricing Strategy is a straightforward approach. For instance, if the production cost of a dish like Chicken Biryani is ₹100, and your target profit margin is 50%, the pricing calculation would be: ₹100 (cost) + (₹100 x 0.5 [50% profit]) = ₹150. This method ensures a well-defined profit margin above the production cost, greatly assisting in effective financial management.
In addition, it is imperative to complement this strategy with a comprehensive understanding of market dynamics and customer preferences.

Set Pricing Strategies according to the restaurant type
A recent analysis shows the impact of menu pricing on restaurant performance. A mere 1% increase in food item prices can lead to a 0.27% increase in footfall. 
Customers look to higher menu prices as a sign of higher quality and value and are more willing to visit the restaurant.
Before setting prices for your restaurant menu, it’s crucial to remember the type of restaurant you operate. Your pricing should include the following factors: raw materials, labor, interior decor, maintenance, and miscellaneous expenses. The target is to ensure the menu price reflects your food cost, overhead, labor, and desired profit for each item.
To effectively manage this, you can calculate your Gross Margin Value (GMV): 
GMV=(Total Revenue earned – Cost of Goods Sold) /Revenue 
Aiming for a GMV between 60-65% is typical, but this can vary based on the type of restaurant. Different restaurant formats have varying overheads and service levels, influencing their GMV:
Fine Dining Restaurants – 75%.
Casual Dining Restaurants – 55%.
Quick Service Restaurants (QSRs) – 45%.

Premium Pricing Strategy for Exotic Cuisine 
With the rising trend of exotic cuisines in India, as highlighted by a striking 163.17% growth in interest for Korean cuisine, followed by Chinese, Vietnamese, Mexican, and Turkish cuisines, Indian restaurants can capitalize on this trend through their pricing strategies.
Embracing this popularity, if your restaurant specializes in exotic cuisines, it is strategic to set premium prices. This trend is not merely a reflection of the cost of ingredients but a representation of your unique culinary experience. For example, if your restaurant serves authentic Korean dishes, pricing them higher than local Indian cuisines can significantly elevate their perceived value. This premium pricing creates a psychological impression of an exclusive and refined dining experience.
Consider a dish like Bibimbap or Kimchi Jjigae; pricing these dishes higher than typical Indian offerings communicates their exclusivity and the specialized culinary skill involved in their preparation. This strategy is about the food and the entire cultural and gastronomic journey that the diner experiences.

Conclusion 
Restaurant owners must understand these essential strategies to enhance customer engagement and increase profitability. It focuses on adopting digital menus, applying relative pricing, and implementing a cost-plus system, all tailored to fulfill the diverse needs of your customers. Recognizing the significance of setting prices based on your restaurant’s type and exploring the potential of premium pricing for exotic cuisines are crucial steps. To maintain a competitive edge and effectively refine your menu pricing strategies, consider integrating innovative solutions like those Restroworks offers.

Frequently Asked Questions
1. What is Menu Engineering and How Does it Relate to Digital Menus?
Menu engineering is a strategy to maximize revenue by identifying high-profit items and positioning them strategically on digital menus. This approach drives customer choices towards these dishes, enhancing profitability. For example, placing regular fries (with high margins) next to pricier cheesy fries can lead customers to opt for the more profitable regular fries​​.
2. How Does Cost-Based Pricing Work in a Digital Menu Context?
Cost-based pricing involves adding a fixed markup to the cost of each dish, ensuring profitability. In a digital menu setup, this can be dynamically adjusted to reflect real-time costs. For example, if a dish costs 55 SAR to produce and a 30% markup is added, the menu price would be 71.5 SAR. Digital menus allow for quick updates to these prices based on fluctuating costs​​.
3. What Role Does Value-Based Pricing Play in Digital Menu Strategies?
Value-based pricing assesses the perceived value of offerings, setting higher prices for dishes with exceptional quality or uniqueness. Digital menus can highlight these unique aspects through visual elements and descriptions, justifying higher prices. For example, specialty dishes made with exotic ingredients can be priced higher to reflect their unique value compared to standard menu items​​.

Restroworks is a leading platform that specializes in providing technological solutions to the restaurant industry. It stands out for its ability to streamline operations, enhance customer experiences, and enable scalability for global restaurant chains.

### agent 3: task_output.agent]
# Market Research Report: Trends in Menu Design, AI in Restaurants, and Pricing Strategies

## Executive Summary

As the restaurant industry continues to evolve in a technology-driven era, digital transformation plays a critical role in reshaping customer experiences and operational efficiencies. This report delves into the current trends in menu design, the integration of AI in restaurant operations, and innovative pricing strategies that are essential for staying competitive in 2024. By leveraging these insights, restaurant owners can enhance customer engagement, optimize profitability, and streamline operations.

## Trends in Menu Design

### Digital Menu Boards in 2024

1. **Integration with Back-Office Systems**: Digital menu boards are becoming increasingly sophisticated, with deeper integration with back-office data and POS systems. This real-time update capability ensures operational efficiency and accuracy in menu displays.

2. **Analytics-Driven Marketing**: The strategic use of analytics is set to optimize menu marketing by understanding customer preferences and behaviors. Enhanced analytics allow for more targeted marketing campaigns on digital menu boards.

3. **Affordable Technology**: The decreasing cost of digital menu board technology makes it more accessible for smaller establishments, lowering the initial investment barrier.

4. **Ease of Management**: Advanced management tools, such as SmarterSign, simplify the creation and control of digital menu boards, enabling rapid updates without the need for design experience.

5. **AI-Enhanced Dynamic Presentations**: AI integration is leading to real-time adaptability in digital menu boards, with personalization based on customer demographics, purchase history, and external data like weather conditions, thus enhancing customer engagement and sales.

## AI in Restaurants

AI's integration into restaurant operations is transforming how businesses interact with customers and manage their resources. Digital menu boards, enhanced with AI, adapt dynamically to provide tailored customer experiences and drive higher engagement. AI facilitates predictive analytics, enabling restaurants to anticipate customer preferences and optimize menu offerings.

## Pricing Strategies for 2024

### Leveraging Digital Menus

- **Enhanced Customer Engagement**: Digital menus are transforming the traditional restaurant experience into a more interactive model, improving order accuracy and tracking, and enabling real-time updates.
- **Case Study Insight**: Chili's implementation of digital tablets resulted in a 20% increase in appetizer and dessert sales, showcasing the potential for revenue growth.

### Relative Pricing Strategies

This strategy involves positioning high-margin items alongside pricier options to make them appear more attractive. For instance, within a diversified menu, high-margin dishes like Paneer Butter Masala can be strategically placed to drive sales and enhance overall revenue by 4.5%.

### Cost-Plus Pricing

A straightforward approach that involves adding a profit margin to the production cost of dishes. This ensures a consistent profit margin, aiding in effective financial management.

### Restaurant-Specific Pricing

Pricing strategies should reflect the type of restaurant, considering factors like raw materials, labor, and overheads. Different formats, such as fine dining or quick service, have varying overheads that influence pricing strategies.

### Premium Pricing for Exotic Cuisines

With growing interest in exotic cuisines, restaurants can capitalize on this trend by setting premium prices, which reflect not only the cost of ingredients but also the unique culinary experience. This strategy enhances the perceived value of exclusive cuisines.

## Actionable Recommendations

1. **Adopt Digital Menu Boards**: Integrate digital menu boards with back-office and POS systems to improve operational efficiency and enhance customer experience.
   
2. **Utilize AI for Personalization**: Incorporate AI to provide dynamic, personalized menu presentations, leveraging customer data and analytics for targeted marketing.

3. **Implement Innovative Pricing Strategies**: Use relative and cost-plus pricing to cater to diverse customer preferences and optimize profit margins. Consider the type of restaurant when setting prices to align with customer expectations and business goals.

4. **Explore Premium Pricing Opportunities**: Respond to the rising demand for exotic cuisines by setting premium prices that reflect the unique value of these offerings and enhance customer perception.

By embracing these trends and strategies, restaurants can maintain a competitive edge, foster greater customer loyalty, and achieve sustained growth in a rapidly evolving market.